In [1]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import sqlite3
import re
from collections import defaultdict

def read_dictionary(filename):
    """
    Opens the indicated filename and creates a list line by line using the first entry on each line
    as the key and the second as the value
    """
    dictionaryoutput = {}
    with open(filename) as file:
        entries = csv.reader(file)
        for item in entries:
            dictionaryoutput[item[0]] = item[1]
    return dictionaryoutput

def write_dict(outputfilename, dictionary):
    """
    Writes a dictionary to a csv.
    """
    # May want to modify this code to pickle the key and value and alter the read dictionary to do the same.

    with open(outputfilename, 'w', newline='') as outfile:
        for key, value in dictionary.items():
            outfile.write('%s,%s\n' % (key, value))
            
def write_list(outputfilename, list):
    """
    Writes a list to a csv.
    """
    with open(outputfilename, 'w', newline='') as outfile:
        itemwriter = csv.writer(outfile, delimiter=",")
        for item in list:
            itemwriter.writerow(item)
            
def read_list(filename):
    """
    Opens the indicated filename and creates a list line by line
    """
    listoutput = []
    with open(filename) as file:
        entries = csv.reader(file)
        for item in entries:
            listoutput.append(item)
    return listoutput

ABRDict = read_dictionary('C:\\Dropbox\\Dropbox\\HAXz\\CBBTO\\espn_scraper\\ABRdictionary.csv')
NUMDict = read_dictionary('C:\\Dropbox\\Dropbox\\HAXz\\CBBTO\\espn_scraper\\NUMdictionary.csv')


In [2]:
conn = sqlite3.connect('C:\\Dropbox\\Dropbox\\HAXz\\CBBTO\\data\\CBBdb.sqlite3')
curr = conn.cursor()

# Exclude bad data
# It might be possible to salvage some of these games
data = pd.read_sql_query("""SELECT rowid,*
                        FROM playbyplay
                        WHERE game_id not in ('253240066', '253510006', '260692440', '283190221',
                        '283192166', '283190201', '293170264', '283342390', '293172633', '303172239', '290450275',
                        '280472483', '283472390', '290450275', '290650087', '293170041', '260280218', '263240150', 
                        '270170222', '400498526', '400591228', '273332603', '310630171', '253190235', '253532250', 
                        '260020150', '253180142', '253340120', '253422250', '260072608', '260120204', '260230221',
                        '260282507', '260282294', '260310275', '260360152', '263240096', '263330221', '263330264',
                        '263400057', '263572305', '270090275', '270130062', '270152305', '270150097', '270160254',
                        '270232390', '270272305', '270380201', '270460012', '270462509', '270550079', '270672440',
                        '270670120', '273212501', '273512250', '273532086', '280030305', '280050103',                        280430258, 280562306, 280590097, 280612181, '280612181', '280660356', '280730235', '283310153',
                        '283402181', '283420052', '290092561', '290350026', '290452752', '290490235',
                        '290530183', '290592550', '290660277', '293130153', '293160194', '293172509', '293200041',
                        '293290012', '293392509', '293410251', '293540059', '300090167', '300300096', '303120356',
                        '303160127', '303160062', '303170068', '303172181', '303170036', '303180150', '303180084',
                        '303200097', '303250152', '303300222', '303310251', '303320127', '303360239', '303390030',
                        '303452350', '303480251', '303500142', '303520082', '303580326', '310052633', '310070315',
                        '310090150', '310122599', '310120046', '310120142', '310150145', '310200264', '310350163',
                        '310360097', '310562099', '313160275', '313310221', '313500140', '320140077', '320490305',
                        '323280057', '323300038', '323330156', '330012506', '330712065', '400498358', '400502811',
                        '400507286', '400545134', '400597751', '400589302', '400589302', '400585776', '400586395',
                        '400835654', '400869965')""", conn)



conn.close()

# Ensure that numbers are numeric values and not strings.
data = data.apply(pd.to_numeric,errors='ignore')


In [ ]:
data.head(2)

,rowid,game_id,date,event_index,time,actor,event,away,home,away_score,home_score
0,1,253120183,2005-11-09,0,1194,183,Darryl Watkins missed Two Point Layup.,BCU,SYR,0,0
1,2,253120183,2005-11-09,1,1176,183,Louie McCroskey made Three Point Jumper.,BCU,SYR,0,3


In [ ]:
# We need to divide games into periods so that we don't span halftime,
# overtime periods or different games.

# Add each half/ot to periods list
periods = []
lastindex = 0
lasttime = data.iloc[0]['time']
for i, row in data.iterrows():
    time = row['time']
    if time > lasttime + 1:
        period = data[lastindex:i]
        period['game_id'].values[0]
        periods.append(data[lastindex:i])
        lastindex = i
    lasttime = time
    
#Add the last period
periods.append(data[lastindex:i])

In [ ]:
# This method is a lot more sensitive to out of order events
bad_games = []
game_sequences = []
last_game_id = -1
for period in periods:
    game_id = period.game_id.values[0]
    # Should be O(n) to add to strings
    # https://stackoverflow.com/questions/4435169/good-way-to-append-to-a-string
    game_sequence = ''
    # We carry the halftime/overtime score and reset on a new game 
    if game_id != last_game_id:
        last_home = 0
        last_away = 0
    for i,row in period.iterrows():
        # Excluding last 90 seconds
        if row.time < 90:
            break
        points = (row.home_score - last_home) - (row.away_score - last_away)
        # We need to make sure that this is accurate, otherwise 2 digit scores throw off the analysis
        if points > 3 or points < -3:
            # This should be empty, so we aren't going to continue or break
            bad_games.append(row.game_id)
        last_home = row.home_score
        last_away = row.away_score
        if 'timeout' in row.event.lower():
            if 'tv' in row.event.lower():
                game_sequence += 'OT'
            if row.actor != -1:
                if NUMDict[str(row.actor)] in ABRDict[row.home]:
                    game_sequence += 'HT'
                elif NUMDict[str(row.actor)] in ABRDict[row.away]:
                    game_sequence += 'AT'
            else:
                 game_sequence += 'UT'
        elif points > 0:
            game_sequence += 'H' + str(points)
        elif points < 0: 
            game_sequence += 'A' + str(abs(points))
    # Get current values so we can make sure we have the correct starting score 
    last_game_id = game_id
    last_home = period.iloc[-1].home_score
    last_away = period.iloc[-1].away_score
            
    game_sequences.append([game_id, game_sequence])

# Check a random value to make sure it's correct
print(game_sequences[123])
print(bad_games)

In [ ]:
# Some of the above is a bit slow so we can read/write here
#write_list('game_sequences.csv', game_sequences)
#game_sequences = read_list('game_sequences.csv')

In [ ]:
# We need to use collections.defaultdict to append list items to a dictionary key 
# See Python Docs

full_run_dictionary = defaultdict(list)
for period in game_sequences:       
    for start,end in [(m.start(0), m.end(0)) for m in re.finditer(r"H[1-3][H1-3]+", period[1])]: # Make sure we get at least 2 baskets
        run = period[1][start:end]
        sequence = period[1][end:end+10]  # Surprising that this hasn't thrown an error yet?
        try:
            full_run_dictionary[run].append(sequence)
        except:
            full_run_dictionary[run] = [sequence]    
    for start,end in [(m.start(0), m.end(0)) for m in re.finditer(r"A[1-3][A1-3]+", period[1])]:
        run =  period[1][start:end]
        sequence = period[1][end:end+8]
        try:           
            full_run_dictionary[run].append(sequence)
        except:
            full_run_dictionary[run] = [sequence]
            

print(full_run_dictionary)

In [ ]:
# Here we find out how much each sequence is worth and the scores after the timeout
# We produce a list of [run_sequence, value of run, [values of post TO sequences]]
TO_run_count = 0
other_count = 0
score_histograms = []
for key in full_run_dictionary:
    run_value = 0
    for c in str(key):
        if c in 'OUT':
            print('=========ERROR========')
        elif c == 'A':
            sign = -1
        elif c == 'H':
            sign = 1
        else:
            run_value += sign * int(c)
    if run_value == 0:
        print(key)
    score_histograms.append([key, run_value])
    seq_values = []
    to_seq_values = []
    h_seq_values = []
    a_seq_values = []
    o_seq_values = []
    TO_run = False
    for seq in full_run_dictionary[key]:
        TO_run = 'T' in str(seq)[:2]
        try:
            caller = str(seq)[0]
        except:
            # Empty list
            continue
        seq_value = 0
        to = False
        for c in str(seq):
            # We are going to set the value of turnovers to 0 because we
            # want to be able to subtract the last item so TO runs and non-TO runs
            # have the same length
            if to == True:
                c = 0
                to = False
            elif c in 'OUT':
                to = True
                c = 0
                continue
            elif c == 'A':
                sign = -1
            elif c == 'H':
                sign = 1
            else:
                seq_value += sign * int(c)
        if TO_run:
            to_seq_values.append(seq_value)
            if caller == 'A':
                a_seq_values.append(seq_value)
            elif caller == 'H':
                h_seq_values.append(seq_value)
            elif caller == 'O':
                o_seq_values.append(seq_value) 
            # Not using else because I still need to fix the bad actor entires (i.e., -1)
        else:
            # We knoew for a fact that the first event after the other sequences are TOs
            # and that the first even after this sequence isn't a TO, so we want to use the same number of events
            # Might be worth identifying more timeout situations
            try:
                seq_values.append(seq_value - sign * int(c))
            except:
                print(seq)

    score_histograms[-1].append([seq_values, to_seq_values, a_seq_values.append(seq_value), h_seq_values.append(seq_value), o_seq_values.append(seq_value)])
        
print(len(score_histograms))

In [ ]:
for elm in score_histograms:
    print(len(elm[2][0]))

In [ ]:
# Plot histograms for all of different found sequences
for gram in score_histograms:
    plt.hist(gram[2][0], bins=10, alpha=0.5, label='No TO')
    plt.hist(gram[2][1], bins=10, alpha=0.5, label='TO')
    plt.hist(gram[2][2], bins=10, alpha=0.5, label='Away TO')
    plt.hist(gram[2][3], bins=10, alpha=0.5, label='Home TO')
    plt.hist(gram[2][4], bins=10, alpha=0.5, label='Official TO')
    plt.legend(loc='upper right')
    plt.title(gram[0] + ' -- Run Value: ' + str(gram[1]))
    plt.show()

In [ ]:
min_run = 0
max_run = 0
for extremes in score_histograms:
    if extremes[1] < min_run:
        min_run = extremes[1]
    if extremes[1] > max_run:
        max_run = extremes[1]
combined_by_value = [[] for i in range(max_run - min_run+1)]

for record in score_histograms:
    combined_by_value[record[1]-min_run].extend(record[2])
        

In [ ]:
h_values = []
h_averages = []
a_values = []
a_averages = []
for i,histo in enumerate(combined_by_value):
    if len(histo) == 0:
        continue
    
    plt.hist(histo, bins=15, color='r', alpha=1, label='diff')
    plt.title('Run Value: ' + str(i+min_run) + ' -- Average Post Timeout: ' + '{:.2f}'.format(sum(histo)/len(histo)))
    plt.show()
    
    if i+min_run > 0 and i+min_run < 15:
        h_values.append(i+min_run)
        h_averages.append(sum(histo)/len(histo))
    elif i+min_run < 0: # Don't want i+min_run > 15
        a_values.append(i+min_run)
        a_averages.append(sum(histo)/len(histo))

In [ ]:
plt.scatter(h_values, h_averages)
plt.show()

In [ ]:
plt.scatter(a_values, a_averages)
plt.show()

In [ ]:
#data.loc[(data.game_id == 253120183)]

In [ ]:
# Nice little alert to let me know when the Notebook has finished
from IPython.display import HTML
HTML('''<script>
alert('Done!')
</script>''')